# CROP YIELD DATA

## First I will combine all crop yield data into one DataFrame (yield_df) and import that table into Postgres to filter across all 7 data sets. Then I will pull those SQL queries back here to make visualizations.
- This can all be done in Python but I want to practice more SQL and get used to how SQL and Python connect to each other

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import requests as re
import glob
import os

Import multiple CSVs and concate into one DataFrame:
https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

In [2]:
path = r'C:\Users\Jasmine\Documents\NSS\Projects\AgAidFromAbroad\data_yields' # use your path
all_files = glob.glob(os.path.join(path, "*.csv")) # advisable to use os.path.join as this makes concatenation OS independent

df_from_each_file = (pd.read_csv(f) for f in all_files)
yield_df   = pd.concat(df_from_each_file, ignore_index=True)

In [ ]:
# Check all files imported into one
yield_df

In [5]:
# Save it to the same folder
yield_df.to_csv(r'C:\Users\Jasmine\Documents\NSS\Projects\AgAidFromAbroad\data_yields\yield_all.csv', index=False)

I will analyze this master crop yield CSV in SQL and replicate the queries to below

### How does the number of TLC approvals compare to the number of H-2A visas issued?